In [14]:
from labic_images_segmentation import Dataset, DataAugmentation, SegmentationModel, SaveReport, PredictImages, DiceCoef
import tensorflow as tf

In [15]:
dataset_folder = "./TM40_Original"
norm_imgs_folder = "/Norm_images/*"
gt_folder = "/GT_images/*"

output_folder = "./TM40_46Prod"
test_imgs_folder = "/Producao/*"
gt_test_folder = "/GT_Producao/*"

ORIGINAL_SIZE = 256
NEW_SIZE = 128

n_fold = 1
batch_size = 4
epochs = 2

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=0, patience=50)

In [16]:
imgs = Dataset(folder=dataset_folder, norm_imgs_folder=norm_imgs_folder, gt_folder=gt_folder, NEW_SIZE=NEW_SIZE)


Imagens carregadas com sucesso.


In [17]:
imgs.split_dataset()


Dataset subdividido.
20.0% dos dados para validação.
80.0% dos dados para treino.


In [18]:
data_aug = DataAugmentation(imgs.X_train, imgs.Y_train, imgs.X_val, imgs.Y_val)


Etapa de Data Augmentation concluída.


In [19]:
segment = SegmentationModel(X_train=imgs.X_train, trainDS=data_aug.trainDS, valDS=data_aug.valDS, epochs=epochs, callback=early_stopping)

Epoch 1/2
233/233 [==============================] - 12s 42ms/step - loss: 0.3757 - iou_score: 0.6818 - val_loss: 0.9729 - val_iou_score: 0.2971
Epoch 2/2
233/233 [==============================] - 9s 40ms/step - loss: 0.1129 - iou_score: 0.9006 - val_loss: 0.2216 - val_iou_score: 0.8184


In [20]:
save_report = SaveReport(model=segment.model, folder_name=output_folder, n_fold=n_fold, use_batch_size=batch_size, epochs=epochs)

Diretorio  ./TM40_46Prod/outputs/Exec_2023-04-16-18-32-18.025849  ja existe
Diretorio  ./TM40_46Prod/outputs/Exec_2023-04-16-18-32-18.025849/fold_1  Criado 

Modelo salvo.
Nome: model_4_2_exec_2023-04-16-18-32-18.025849_fold_1
Salvo em: ./TM40_46Prod/outputs/Exec_2023-04-16-18-32-18.025849


In [21]:
test_imgs = Dataset(folder=output_folder, norm_imgs_folder=test_imgs_folder, gt_folder=gt_test_folder, NEW_SIZE=NEW_SIZE)


Imagens carregadas com sucesso.


In [22]:
prediction = PredictImages(test_images=test_imgs, n_fold_folder_name=save_report.n_fold_folder_name, \
                           model_name=save_report.model_name, use_batch_size=batch_size, img_shape=test_imgs.img_shape)

Diretorio  ./TM40_46Prod/outputs/Exec_2023-04-16-18-32-18.025849/fold_1/outputs_prod  Criado 


c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:237: UserWarning: ./TM40_46Prod/outputs/Exec_2023-04-16-18-32-18.025849/fold_1/outputs_prod/predicao_025_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7:-4]), str(self.batch)),\
c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:237: UserWarning: ./TM40_46Prod/outputs/Exec_2023-04-16-18-32-18.025849/fold_1/outputs_prod/predicao_026_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7:-4]), str(self.batch)),\
c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:237: UserWarning: ./TM40_46Prod/outputs/Exec_2023-04-16-18-32-18.025849/fold_1/outputs_prod/predicao_027_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7


Imagens preditas com sucesso.


c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:237: UserWarning: ./TM40_46Prod/outputs/Exec_2023-04-16-18-32-18.025849/fold_1/outputs_prod/predicao_404_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7:-4]), str(self.batch)),\
c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:237: UserWarning: ./TM40_46Prod/outputs/Exec_2023-04-16-18-32-18.025849/fold_1/outputs_prod/predicao_405_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7:-4]), str(self.batch)),\
c:\Yana\girino_att_Yana\girino-segmentation\labic_images_segmentation.py:237: UserWarning: ./TM40_46Prod/outputs/Exec_2023-04-16-18-32-18.025849/fold_1/outputs_prod/predicao_406_4.png is a low contrast image
  io.imsave(self.n_fold_folder_name + '/outputs_prod/predicao_%s_%s.png'%(str(self.test_images.GT_imgs[i][-7

In [23]:
prediction.new_predicao.shape

(408, 128, 128, 1)

In [24]:
test_imgs.Y.shape

(408, 128, 128, 1)

In [25]:
prediction.n_fold_folder_name

'./TM40_46Prod/outputs/Exec_2023-04-16-18-32-18.025849/fold_1'

In [26]:
DiceCoef(gt_imgs=test_imgs.Y, pred_folder=prediction.n_fold_folder_name+"/outputs_prod/*", new_size=NEW_SIZE)

Coeficiente Dice: 0.9264891428187463
